(techniques:compositional_se)=
# Compositional Symbolic Execution

In [1]:
import utils
from minipy import *
from semantics import *
from symbolic_interpreter import *
from transparent_function_execution import *
from static_loops import *

Symbolic execution is a powerful program exploration technique, useful both in program verification, where SE overapproximates all feasible program paths, and test case generation, where SE steers the next program execution towards a yet uncovered path. In both application scenarios, however, it suffers from the *path explosion problem*: There are simply too many feasible program paths in large, realistic programs. This leads to immense symbolic execution trees in the case of static SE, and very long, complicated path conditions to be processed in the case of dynamic/concolic SE.

One solution to this problem is to render symbolic execution into a *compositional* analysis by analyzing not complete systems, but individual functions one at a time. This is accomplished by annotating functions with *summaries*, which conjoin "constraints on the function inputs observed during the exploration
of a path (...) with constraints observed on the outputs" {cite}`baldoni.coppa.ea-18`. Instead of symbolically executing a called function, we can use its *summary* to obtain the resulting symbolic state, which can dramatically reduce the overall search space.

Function summaries seem to have arisen independently in the areas of SE-based test generation and program verification. In the former area, Godefroid {cite}`godefroid-07` introduces the idea, building on existing similar principles form interprocedural static analysis (e.g., {cite}`reps.horwitz.ea-95`). As natural in automated test case generation, summaries are expected to be computed automatically; they are means for re-using previously discovered analysis results. In {cite}`anand.godefroid.ea-08`, the original idea of function summaries is extended to a demand-driven approach allowing lazy expansion of of incomplete summaries, which further improves the performance of the analysis.

For the area of program verification, we did not find an explicit reference to the first original work using function summaries for modular, static symbolic execution. However, function summaries are already mentioned as a means for modularization in the first paper reporting on the KeY project {cite}`ahrendt.baar.ea-00`, which appeared in 2000. Usually, function summaries are called "contract" in the verification context, inspired by the "design-by-contract" methodology {cite}`meyer-92`. Contracts are not only used for scalability, but additionally define the verification goals, i.e., the expected behavior, of functions in the program under test.

To support compositional symbolic execution, we extend our framework in two directions:

1. We add a convenience method to the symbolic interpreter which allows executing an individual function by its name, even without providing an initial symbolic environment.
2. We implement a new code transformer which replaces function calls by assertions of preconditions, `havoc` statements resetting the value of the variable to which the result of the function call is assigned, and assumptions of postconditions. This is similar to the transformation for SE with loop invariants described in {ref}`techniques:loops_static_se`, only that we do not verify that a function respect its contract when evaluating calls to that function. The verification can be done separately, for instance by suitably placed `assert` statements.

```{admonition} TODO
- Add both extensions describe above
- Introduce insertion sort example below
- Transform example, compositionally execute all functions. Maybe also non-compositionally to demonstrate how cumbersome this is.
```

In [2]:
def insertion_point(x: int, t: tuple) -> int:
    i = 0
    while i < len(t):
        if t[i] >= x:
            break

        i = i + 1

    return i


def insert(elem: int, at: int, t: tuple) -> tuple:
    assert 0 <= at <= len(t)

    result = tuple()
    i = 0
    while i < at:
        result = result + (t[i],)
        i = i + 1

    result = result + (elem,)

    while i < len(t):
        result = result + (t[i],)
        i = i + 1

    return result


def sort(t: tuple) -> tuple:
    result = tuple()

    i = 0
    while i < len(t):
        elem = t[i]
        pos = insertion_point(elem, result)
        result = insert(elem, pos, result)
        i = i + 1

    return result

## References

```{bibliography}
:filter: docname in docnames
```